In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import fastai.basics as fai
import fastai.vision as fv
import fastai.callbacks.hooks as fch
import numpy as np
from pathlib import Path
import gc

In [ ]:
path = fai.untar_data(fai.URLs.CAMVID)

In [ ]:
path.ls()

In [ ]:
images = (path/"images").ls()
labels = (path/"labels").ls()

In [ ]:
!cat {path}/valid.txt | head

In [ ]:
img_file = images[0]
img = fv.open_image(img_file)
img

In [ ]:
img_file = labels[2]
img = fv.open_mask(img_file) # cambia por "open_mask"
img

In [ ]:
def get_label(x):
    return path/'labels'/f'{x.stem}_P{x.suffix}'

In [ ]:
!cat {path/'codes.txt'}

In [ ]:
codes = np.loadtxt(path/'codes.txt', dtype=str); codes

In [ ]:
codes[2]

In [ ]:
size = np.array(img.shape[1:]); size

In [ ]:
def load_data(img_size, batch_size):
    tfms = fv.get_transforms()
    data = (fv.SegmentationItemList
            .from_folder(path/"images")
            .split_by_fname_file(path/"valid.txt")
            .label_from_func(get_label, classes=codes)
            .transform(tfms, size=img_size, tfm_y=True)
            .databunch(bs=batch_size)
            .normalize(fv.imagenet_stats))
    return data

In [ ]:
data = load_data(size//2, 4)

In [ ]:
data.show_batch(2)

In [ ]:
name2id = {v:k for k,v in enumerate(codes)}

In [ ]:
name2id

In [ ]:
void_code = name2id['Void']

In [ ]:
def acc_camvid(input, target):
    target = target.squeeze(1)
    mask = (target != void_code)
    return (input.argmax(dim=1)[mask] == target[mask]).float().mean()

In [ ]:
learn = fv.unet_learner(data, fv.models.resnet34, metrics=acc_camvid, wd=1e-2, bottle=True)

In [ ]:
print(learn.summary())

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, slice(1e-3), pct_start=0.8)

In [ ]:
learn.save('stage-1')

In [ ]:
gc.collect()

In [ ]:
learn.unfreeze()

In [ ]:
learn.save('stage-1-big')

In [ ]:
learn.validate()

In [ ]:
learn.show_results(rows=3)